# WORK IN PROGRESS

## Import libraries

In [1]:
import os
import sys

import pandas as pd
import geopandas as gpd
import osmnx as ox
import numpy as np

from scipy.spatial import Voronoi, voronoi_plot_2d
import shapely

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/home/jovyan/accesibilidad-urbana/aup/data.py:25: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(


## Work in progress based on Script 03-population_to_nodes

In [ ]:
# gdf_mun
metro_schema = 'metropolis'
metro_table = 'metro_gdf_2020'

query = f"SELECT * FROM {metro_schema}.{metro_table}"
metro_gdf = aup.gdf_from_query(query, geometry_col='geometry')
metro_gdf = metro_gdf.set_crs("EPSG:4326")

city_list = list(metro_gdf.city.unique())

# Show
print(metro_gdf.shape)
print(len(city_list))
print(city_list)
metro_gdf.head(1)

In [ ]:
censo_column_start = 14 #column where numeric data starts in censo (16 for 2010, 14 for 2020)
censo_column_end = 0 #column where numeric data ends in censo (-1 for 2010, all up to the end (0) for 2020)
year = '2020'
pop_schema = 'censoageb'
pop_table = 'censoageb_' + year

res_list = [8] #Only 8,9,10 and 11 available

save_schema = 'censo'
save_table = f'censo_inegi_{year[:2]}_ageb_hex'

In [ ]:
def main():
    # Load city
    city_gdf = metro_gdf.loc[metro_gdf.city == city]

    # Load agebs of city
    ageb_gdf = gpd.GeoDataFrame()
    for cvegeo in list(city_gdf.CVEGEO.unique()):
        query = f"SELECT * FROM {pop_schema}.{pop_table} WHERE \"cve_geo\" LIKE \'{cvegeo}%%\'"
        ageb_gdf = pd.concat([ageb_gdf,aup.gdf_from_query(query, geometry_col='geometry')])

    # Load hexgrids of city
    hex_bins = gpd.GeoDataFrame()
    for res in res_list:
        # Load hexgrids
        query = f"SELECT * FROM hexgrid.hexgrid_{res}_city_2020 WHERE \"city\" LIKE \'{city}\'"
        hex_bins = pd.concat([hex_bins,aup.gdf_from_query(query, geometry_col='geometry')])    
    hex_bins = hex_bins.set_crs("EPSG:4326")

    return (ageb_gdf,hex_bins)

In [ ]:
k = len(city_list)

# prevent cities being analyzed several times in case of a crash
processed_city_list = []
try:
    query = f"SELECT city FROM {save_schema}.{save_table}"
    cities_processed = aup.df_from_query(query)
    processed_city_list = list(cities_processed.city.unique())
except:
    pass

# PRINT PROGRESS
missing_cities_list = []
for city in city_list:
    if city not in processed_city_list:
        missing_cities_list.append(city)

i = len(processed_city_list)
print(f'Already processed ({i}/{k}) cities.')
print(f'Missing procesing for cities: {missing_cities_list}')
 
# SKIP FAILED CITIES
skip_city_list = []

missing_cities_list = ['Aguascalientes']

for city in missing_cities_list:
    if city not in skip_city_list:
        print("--"*40)
        i = i + 1
        print(f"Starting city {i}/{k}: {city}")
        ageb_gdf,hex_bins = main()
        #main(schema, folder_sufix, year, censo_column_start, censo_column_end, save=True)

In [ ]:
ageb_gdf.nom_mun.unique()

In [ ]:
# city
#metro_schema = 'metropolis'
#metro_table = 'metro_gdf_2020'
#c = 'Aguascalientes'

#query = f"SELECT * FROM {metro_schema}.{metro_table} WHERE \"city\" LIKE \'{c}\'"
#metro_gdf = aup.gdf_from_query(query, geometry_col='geometry')
#metro_gdf = metro_gdf.set_crs("EPSG:4326")
#aoi = metro_gdf.dissolve()

# Show
#print(metro_gdf.shape)
#metro_gdf.head(1)

In [ ]:
# agebs_gdf
#agebs_schema = 'marco'
#agebs_table = 'ageb_2020'

#agebs_gdf = aup.gdf_from_polygon(aoi,agebs_schema,agebs_table)

# Show
#print(agebs_gdf.shape)
#agebs_gdf.head(1)